# Preparando os dados do IPTU de 2022

Para o primeiro experimento vamos preparar os dados do IPTU de 2022, e posteriormente validar com as informações apresentadas no DashBoard

In [1]:
import vaex
import geopandas as gpd
from shapely.validation import make_valid
from difflib import SequenceMatcher
import numpy as np
# import pygeos

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')

gdf_quadras = gpd.read_file('zip://data/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')
gdf_quadras = gdf_quadras.set_crs(epsg=31983)
gdf_quadras = gdf_quadras[gdf_quadras.qd_tipo == 'F']
gdf_quadras = gdf_quadras.dissolve(['qd_setor', 'qd_fiscal']).reset_index()

gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017')
gdf_od_sp = gdf_od[gdf_od.od_municip == '36']

gdf_quadras_distritos = gdf_quadras.sjoin(gdf_distritos, how='left', predicate='intersects')
gdf_quadras_distritos.drop('index_right', axis=1, inplace=True)
gdf_quadras_distritos_od = gdf_quadras_distritos.sjoin(gdf_od_sp, how='left', predicate='intersects')

df_quadras_distritos_od = gdf_quadras_distritos_od.loc[:, ['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub', 'ds_subpref', 'od_id', 'od_nome']]
df_quadras_distritos_od.loc[:, ['sq']] = df_quadras_distritos_od.qd_setor + df_quadras_distritos_od.qd_fiscal
df_quadras_distritos_od.drop_duplicates(keep='first', inplace=True, ignore_index=True)
# df_quadras_distritos_od = vaex.from_pandas(df_quadras_distritos_od)

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [3]:
df_quadras_distritos_od.columns

Index(['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub',
       'ds_subpref', 'od_id', 'od_nome', 'sq'],
      dtype='object')

In [4]:
gdf_od_sp = gdf_od_sp.to_file('data/OD-sp.gpkg', driver='GPKG')

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [5]:
df_quadras_distritos_od

,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sq
0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003
1,001,004,9,BOM RETIRO,09,SE,7,Luz,001004
2,001,005,9,BOM RETIRO,09,SE,7,Luz,001005
3,001,006,9,BOM RETIRO,09,SE,7,Luz,001006
4,001,007,9,BOM RETIRO,09,SE,7,Luz,001007
...,...,...,...,...,...,...,...,...,...
48624,310,117,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310117
48625,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118
48626,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310119
48627,310,120,38,JABAQUARA,15,JABAQUARA,276,Jardim Bom Clima,310120


In [6]:
## TODO
# Verificar essa inconsistencia
gdf_quadras_distritos[gdf_quadras_distritos.ds_codigo.isna()]

,qd_setor,qd_fiscal,geometry,qd_id_orig,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome
33243,156,135,"POLYGON ((343220.581 7387752.345, 343218.992 7...",8985852,F,001,2484.92461758186,1,148338,FISCAL,NaN,NaN,NaN,NaN,NaN
37127,173,160,"POLYGON ((333539.250 7378902.010, 333537.093 7...",9704470,F,001,1348.82739798853,1,182464,FISCAL,NaN,NaN,NaN,NaN,NaN
37132,173,165,"POLYGON ((333600.829 7378409.331, 333597.817 7...",9704234,F,001,2641.57734002054,1,186397,FISCAL,NaN,NaN,NaN,NaN,NaN


In [7]:
dfs = []

for i in range(1995,2023):

    print(f'Processando {i}')
    
    df = vaex.open(f'data/IPTU_{i}/IPTU_{i}.hdf5')
    
    df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
    df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
    df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
    df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
    df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
    df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
    df['sq'] = df.setor + df.quadra
    df['sqlc'] = df.sqlc
    df_sqlc_ac = df.groupby('sqlc', agg={'area_contruida_total': vaex.agg.sum('AREA CONSTRUIDA')})
    df = df.join(df_sqlc_ac, on='sqlc')
    df['fracao_ideal'] = df.func.where(df['FRACAO IDEAL'] == 0.,  
                                                df['AREA CONSTRUIDA'] / df['area_contruida_total'],
                                                df['FRACAO IDEAL'])
    df['ca'] = (df['AREA CONSTRUIDA'] / df['fracao_ideal']) / df['AREA DO TERRENO']
    df['to'] = df['AREA OCUPADA'] / df['AREA DO TERRENO']

    df['valor_terreno'] = df['fracao_ideal'] * df['VALOR DO M2 DO TERRENO'] * df['AREA DO TERRENO']
    df['valor_construcao'] = df['VALOR DO M2 DE CONSTRUCAO'] * df['AREA CONSTRUIDA']

    dfs.append(df)
    # break


Processando 1995
Processando 1996
Processando 1997
Processando 1998
Processando 1999
Processando 2000
Processando 2001
Processando 2002
Processando 2003
Processando 2004
Processando 2005
Processando 2006
Processando 2007
Processando 2008
Processando 2009
Processando 2010
Processando 2011
Processando 2012
Processando 2013
Processando 2014
Processando 2015
Processando 2016
Processando 2017
Processando 2018
Processando 2019
Processando 2020
Processando 2021
Processando 2022


In [8]:
df_iptu = vaex.concat(dfs)

In [9]:
df_iptu.shape

(82322059, 37)

In [10]:
## Parece que para alguns lotes condominiais o terreno está proporcional à fracao ideal
df_iptu[df_iptu.ca > 100]['AREA CONSTRUIDA', 'fracao_ideal', 'AREA DO TERRENO', 'area_contruida_total', 'ANO DO EXERCICIO']

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


#,AREA CONSTRUIDA,fracao_ideal,AREA DO TERRENO,area_contruida_total,ANO DO EXERCICIO
0,67932,1.0,420,67932,1995
1,142,0.004320705918150008,257,32865,2014
2,137,0.004168568385820782,257,32865,2014
3,141,0.004290278411684162,257,32865,2014
4,142,0.004320705918150008,257,32865,2015
...,...,...,...,...,...
237,26,9.999999747378752e-05,2594,16906,2022
238,26,9.999999747378752e-05,2594,16906,2022
239,26,9.999999747378752e-05,2594,16906,2022
240,26,9.999999747378752e-05,2594,16906,2022


In [11]:
df_iptu[df_iptu['NUMERO DO CONDOMINIO'] == '00-0'].shape

(44504583, 37)

In [12]:
df_iptu.shape

(82322059, 37)

In [13]:
df_quadras_distritos_od['sqod'] = df_quadras_distritos_od.sq + df_quadras_distritos_od.od_id.str.pad(width=3, side='left', fillchar='0')

In [14]:
df_quadras_distritos_od.shape

(48629, 10)

In [15]:
df_quadras_distritos_od = df_quadras_distritos_od.drop_duplicates(subset=['sq'])

In [16]:

df_iptu = df_iptu.join(vaex.from_pandas(df_quadras_distritos_od), on='sq', how='left',  
            allow_duplication=False, inplace=False)

In [17]:
df_iptu.shape

(82322059, 46)

## Adicionando informações de Uso

In [18]:
df_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()
df_usos_2022

,TIPO DE USO DO IMOVEL,count
0,Residência e outro uso (predominância residenc...,85643
1,"Outras edificações de uso comercial, com utili...",22163
2,Posto de serviço,2279
3,"Prédio de apartamento, não em condomínio, de u...",2209
4,"Estação radioemissora, de televisão ou empresa...",495
5,Flat de uso comercial (semelhante a hotel),24619
6,Apartamento em condomínio,1439450
7,Garagem (unidade autônoma) em edifício em cond...,200254
8,Clube esportivo,573
9,"Outras edificações de uso especial, com utiliz...",1263


In [19]:
df_usos = df_iptu.groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()

In [20]:
df_usos.fillna('', inplace=True)

In [21]:
results, ratios = [], []

for i, row in df_usos.iterrows():

    similaridade = df_usos_2022['TIPO DE USO DO IMOVEL'].apply(lambda x: SequenceMatcher(None, row['TIPO DE USO DO IMOVEL'], x).ratio())
    
    results.append(df_usos_2022.iloc[similaridade.idxmax()]['TIPO DE USO DO IMOVEL'])

    ratios.append(similaridade.max())

In [22]:
df_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [23]:
residencial = ['residencial', 'residência', 'apartamento', 'cortiço']
comercial = ['comércio', 'comercial', 'posto', 'loja']
servicos =  ['hotel', 'radioemissora', 'flat', 'serviço', 'clube', 'oficina', 'hospital', 'templo', 'escritórios', 'consultórios', 'escritório', 'consultório', 'asilo', 'creche', 'prédio de garagens', 'escola']
industrial = ['Indústria', 'armazéns']

In [24]:
class_residencial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(residencial)), case=False)
class_comercial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(comercial)), case=False)
class_servicos = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(servicos)), case=False)
class_industrial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(industrial)), case=False)

df_usos.loc[class_residencial, ['classificacao_abrangente']] = 'Residencial'
df_usos.loc[class_comercial, ['classificacao_abrangente']] = 'Comercial'
df_usos.loc[class_servicos, ['classificacao_abrangente']] = 'Serviços'
df_usos.loc[class_industrial, ['classificacao_abrangente']] = 'Industrial'

df_usos.loc[df_usos.classificacao_abrangente.isna(), ['classificacao_abrangente']] = 'Outros'

In [25]:
# df_iptu['TIPO DE USO DO IMOVEL']
df_usos = vaex.from_pandas(df_usos.loc[df_usos['TIPO DE USO DO IMOVEL'] != '', ['TIPO DE USO DO IMOVEL', 'classificacao_abrangente']])
df_iptu = df_iptu.join(df_usos, on='TIPO DE USO DO IMOVEL')

In [26]:
df_iptu.shape

(82322059, 47)

## Processando as totalizacoes

In [27]:
gdf_distritos['area'] = gdf_distritos.area
gdf_distritos.to_crs(epsg=4674, inplace=True)

In [37]:
df_iptu['sq'] = df_iptu.qd_setor + df_iptu.qd_fiscal

In [40]:
# df_iptu.sq

In [41]:
# df_iptu.fillna(value='0', column_names=['sq'], inplace=True)
# df_iptu['sq'] = df_iptu['sq'].astype('int')
# df_iptu.categorize('sq', inplace=True)

In [42]:
# df_iptu.head(1)
df_iptu.fillna(value='0', column_names=['ds_codigo'], inplace=True)
df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu.categorize('distrito', inplace=True)
# df_iptu.head(1)

#,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,REFERENCIA DO IMOVEL,CEP DO IMOVEL,QUANTIDADE DE ESQUINAS FRENTES,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,area_contruida_total,NUMERO DO CONDOMINIO,FRACAO IDEAL,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sqod,classificacao_abrangente,distrito
0,0010030001-4,1995,1,--,03812-1,R SAO CAETANO,13.0,--,--,--,01104-001,1,100.0,129.0,1924,1,13.0,loja,comercial horizontal - Padrão B,de esquina,0.0,1963,1,--,135,00-0,1.0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9
1,0010030002-2,1995,1,--,03812-1,R SAO CAETANO,19.0,--,--,--,01104-001,0,100.0,94.0,1944,1,6.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,67,00-0,1.0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9
2,0010030003-0,1995,1,--,03812-1,R SAO CAETANO,27.0,--,--,--,01104-001,0,100.0,94.0,1944,1,8.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,85,00-0,1.0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9
3,0010030004-9,1995,1,--,03812-1,R SAO CAETANO,33.0,--,--,--,01104-001,0,100.0,129.0,1944,1,6.0,loja,comercial horizontal - Padrão B,normal,0.0,1963,1,--,103,00-0,1.0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9
4,0010030005-7,1995,1,--,03812-1,R SAO CAETANO,39.0,--,--,--,01104-001,0,100.0,94.0,1944,1,7.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,98,00-0,1.0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",3101180238-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 42,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38
"82,322,055",3101180239-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 43,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38
"82,322,056",3101180240-3,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 44,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38
"82,322,057",3101190001-4,2022,1,2022-01-01 00:00:00.000000000,19137-0,R COTIPORA,nan,--,CID

In [43]:
# df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu['ano'] = df_iptu['ANO DO EXERCICIO'].astype('int')
df_iptu.categorize('ano', inplace=True)

#,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,REFERENCIA DO IMOVEL,CEP DO IMOVEL,QUANTIDADE DE ESQUINAS FRENTES,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,area_contruida_total,NUMERO DO CONDOMINIO,FRACAO IDEAL,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sqod,classificacao_abrangente,distrito,ano
0,0010030001-4,1995,1,--,03812-1,R SAO CAETANO,13.0,--,--,--,01104-001,1,100.0,129.0,1924,1,13.0,loja,comercial horizontal - Padrão B,de esquina,0.0,1963,1,--,135,00-0,1.0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995
1,0010030002-2,1995,1,--,03812-1,R SAO CAETANO,19.0,--,--,--,01104-001,0,100.0,94.0,1944,1,6.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,67,00-0,1.0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995
2,0010030003-0,1995,1,--,03812-1,R SAO CAETANO,27.0,--,--,--,01104-001,0,100.0,94.0,1944,1,8.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,85,00-0,1.0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995
3,0010030004-9,1995,1,--,03812-1,R SAO CAETANO,33.0,--,--,--,01104-001,0,100.0,129.0,1944,1,6.0,loja,comercial horizontal - Padrão B,normal,0.0,1963,1,--,103,00-0,1.0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995
4,0010030005-7,1995,1,--,03812-1,R SAO CAETANO,39.0,--,--,--,01104-001,0,100.0,94.0,1944,1,7.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,98,00-0,1.0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",3101180238-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 42,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022
"82,322,055",3101180239-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 43,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022
"82,322,056",3101180240-3,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 44,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022
"82,322,057",3101190001-4,2022,1,2022-01-01 

In [44]:
df_iptu.fillna(value='0', column_names=['od_id'], inplace=True)
df_iptu['od'] = df_iptu['od_id'].astype('int')
df_iptu.categorize('od', inplace=True)

#,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,REFERENCIA DO IMOVEL,CEP DO IMOVEL,QUANTIDADE DE ESQUINAS FRENTES,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,area_contruida_total,NUMERO DO CONDOMINIO,FRACAO IDEAL,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sqod,classificacao_abrangente,distrito,ano,od
0,0010030001-4,1995,1,--,03812-1,R SAO CAETANO,13.0,--,--,--,01104-001,1,100.0,129.0,1924,1,13.0,loja,comercial horizontal - Padrão B,de esquina,0.0,1963,1,--,135,00-0,1.0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7
1,0010030002-2,1995,1,--,03812-1,R SAO CAETANO,19.0,--,--,--,01104-001,0,100.0,94.0,1944,1,6.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,67,00-0,1.0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7
2,0010030003-0,1995,1,--,03812-1,R SAO CAETANO,27.0,--,--,--,01104-001,0,100.0,94.0,1944,1,8.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,85,00-0,1.0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7
3,0010030004-9,1995,1,--,03812-1,R SAO CAETANO,33.0,--,--,--,01104-001,0,100.0,129.0,1944,1,6.0,loja,comercial horizontal - Padrão B,normal,0.0,1963,1,--,103,00-0,1.0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7
4,0010030005-7,1995,1,--,03812-1,R SAO CAETANO,39.0,--,--,--,01104-001,0,100.0,94.0,1944,1,7.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,98,00-0,1.0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",3101180238-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 42,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022,275
"82,322,055",3101180239-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 43,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022,275
"82,322,056",3101180240-3,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 44,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022,275
"82,322,057",31

In [45]:
df_iptu.shape

(82322059, 50)

In [46]:
df_iptu.fillna(value='0', column_names=['ds_cd_sub'], inplace=True)
df_iptu['subprefeitura'] = df_iptu['ds_cd_sub'].astype('int')
df_iptu.categorize('subprefeitura', inplace=True)

#,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,REFERENCIA DO IMOVEL,CEP DO IMOVEL,QUANTIDADE DE ESQUINAS FRENTES,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,area_contruida_total,NUMERO DO CONDOMINIO,FRACAO IDEAL,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sqod,classificacao_abrangente,distrito,ano,od,subprefeitura
0,0010030001-4,1995,1,--,03812-1,R SAO CAETANO,13.0,--,--,--,01104-001,1,100.0,129.0,1924,1,13.0,loja,comercial horizontal - Padrão B,de esquina,0.0,1963,1,--,135,00-0,1.0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7,9
1,0010030002-2,1995,1,--,03812-1,R SAO CAETANO,19.0,--,--,--,01104-001,0,100.0,94.0,1944,1,6.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,67,00-0,1.0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7,9
2,0010030003-0,1995,1,--,03812-1,R SAO CAETANO,27.0,--,--,--,01104-001,0,100.0,94.0,1944,1,8.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,85,00-0,1.0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7,9
3,0010030004-9,1995,1,--,03812-1,R SAO CAETANO,33.0,--,--,--,01104-001,0,100.0,129.0,1944,1,6.0,loja,comercial horizontal - Padrão B,normal,0.0,1963,1,--,103,00-0,1.0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7,9
4,0010030005-7,1995,1,--,03812-1,R SAO CAETANO,39.0,--,--,--,01104-001,0,100.0,94.0,1944,1,7.0,loja,comercial horizontal - Padrão A,normal,0.0,1963,1,--,98,00-0,1.0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003007,Comercial,9,1995,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",3101180238-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 42,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022,275,15
"82,322,055",3101180239-1,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 43,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Residencial,38,2022,275,15
"82,322,056",3101180240-3,2022,1,2022-01-01 00:00:00.000000000,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 44,JABAQUARA,BL 15 QD II,04309-011,5,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0,25215,01-9,0.0033,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118275,Resid

In [47]:
# df_iptu_grouped_distrito = \
def agrupamento(campo):
    return df_iptu.groupby([campo, 'ano'],agg={
                        'Quantidade de Unidades': 'count', 
                        'Quantidade de Unidades Condominiais': vaex.agg.count('sqlc',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'Tamanho Médio da Unidade Condominial': vaex.agg.mean('AREA CONSTRUIDA', 
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'Tamanho médio dos Terrenos': vaex.agg.mean('AREA DO TERRENO'),
                        'Área Total dos terrenos-lotes': vaex.agg.sum('AREA DO TERRENO'),
                        'Área Total Ocupada': vaex.agg.sum('AREA OCUPADA'),
                        'Área Total Construída':vaex.agg.sum('AREA CONSTRUIDA'),
                        'Valor Total dos Terrenos': vaex.agg.sum('valor_terreno'),
                        'Valor Total das Construções': vaex.agg.sum('valor_construcao'),
                        'CA médio': vaex.agg.mean('ca'),
                        'TO médio': vaex.agg.mean('to'),
                        'CA médio em lotes condominiais': vaex.agg.mean('ca',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'TO médio em lotes condominiais': vaex.agg.mean('to',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'CA médio em lotes não condominiais': vaex.agg.mean('ca',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] == '00-0'),
                        'TO médio em lotes não condominiais': vaex.agg.mean('to',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] == '00-0'),
                        'Comprimento Médio da Testada': vaex.agg.mean('TESTADA PARA CALCULO'),
                        'Número médio de Pavimentos': vaex.agg.mean('QUANTIDADE DE PAVIMENTOS'),
                        'Fator de obsolecência médio': vaex.agg.mean('FATOR DE OBSOLESCENCIA'),
                        'Residencial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Residencial'),
                        'Comercial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Comercial'),
                        'Serviços': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Serviços'),
                        'Industrial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Industrial'),
                        'Outros': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Outros')

    })

In [48]:
campos = ['distrito', 'subprefeitura', 'od']

In [49]:
dfs = []

for campo in campos:
    print(campo)
    df = agrupamento(campo)
    for u in ['Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros']:
        df[f'Percentual de Uso {u}'] = df[u] / df['Área Total Construída']
    df.export_hdf5(f'data/IPTU-1995-2022-agrupados-por-{campo}.hdf5')
    dfs.append(df)

distrito


/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


subprefeitura
od
sq


In [51]:
# df_iptu_distrito = vaex.open('data/IPTU-1995-2022-agrupados-por-distrito.hdf5')
# df_iptu_subprefeitura = vaex.open('data/IPTU-1995-2022-agrupados-por-subprefeitura.hdf5')
# df_iptu_od = vaex.open('data/IPTU-1995-2022-agrupados-por-od.hdf5')

df_iptu_distrito, df_iptu_subprefeitura, df_iptu_od = dfs

In [52]:
df_iptu_distrito

#,distrito,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos terrenos-lotes,Área Total Ocupada,Área Total Construída,Valor Total dos Terrenos,Valor Total das Construções,CA médio,TO médio,CA médio em lotes condominiais,TO médio em lotes condominiais,CA médio em lotes não condominiais,TO médio em lotes não condominiais,Comprimento Médio da Testada,Número médio de Pavimentos,Fator de obsolecência médio,Residencial,Comercial,Serviços,Industrial,Outros,Percentual de Uso Residencial,Percentual de Uso Comercial,Percentual de Uso Serviços,Percentual de Uso Industrial,Percentual de Uso Outros
0,0,1995,10912,1923,91.49141965678628,3730.6820931085044,40709203,6330766,1474148,403508190.57777524,342587818.0,0.5707595655441189,0.24455748209426265,1.7500919321624053,0.24412733431471892,0.3213535327327567,0.24464950280625053,48.34549120234604,1.6637646627565983,0.6513929618768328,739597,243146,144522,292132,54751,0.5017114970817041,0.1649400195909773,0.09803764615221809,0.19817006162203524,0.037140775553065226
1,0,1996,11178,2100,93.53904761904762,3990.5865986759704,44606777,7943388,1578791,625149057.1666627,476590870.0,0.6077497882852628,0.269002606916139,1.7490463618827914,0.25562280800901827,0.34662696570736623,0.27209773554634026,47.66183574879227,1.7461084272678475,0.675344426552156,814189,251335,150237,302798,60232,0.515704105229888,0.15919459890511156,0.09515952396485665,0.1917910603746791,0.03815071152546474
2,0,1997,10316,2100,93.54,4280.401706087631,44156624,8028760,1728144,667593144.2555504,584468408.0,0.6829332843668249,0.3067563811731817,1.7490600027904148,0.25562280800901577,0.41341707280818957,0.3198260627268294,50.32095773555642,1.9332105467235363,0.7296432725862737,881779,316672,149697,318155,61841,0.5102462526270959,0.18324398892684868,0.0866229897508541,0.1841021350072679,0.035784633687933415
3,0,1998,10300,2117,95.93103448275862,4088.7499029126216,42114124,8054073,1753029,605134057.799998,623670460.0,0.6959584662565197,0.3103514290911142,1.7772406242050165,0.2594796583481374,0.41913014890321987,0.3235123161328903,50.5152427184466,1.9575728155339807,0.7349514563106796,897953,308752,172911,309072,64341,0.5122294040771715,0.1761248673011114,0.09863556164786777,0.17630740849124574,0.03670275848260354
4,0,1999,10350,2232,95.99417562724014,4211.980966183575,43594003,8787774,1809696,609171048.844445,647722993.0,0.7138666176545666,0.3196793561453528,1.743794717411117,0.2742871372307171,0.43361148432587837,0.3321597001484922,50.8751690821256,1.9769082125603865,0.7426086956521739,933763,326504,175967,310033,63429,0.5159778216893887,0.1804192527363712,0.09723566831114176,0.17131772408183474,0.035049533181263595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2,711",96,2018,26008,5980,64.00234113712375,2588.421908643494,67319677,16434666,2579943,1796147331.954893,1946149566.0,0.7594004641919282,0.37377944867127993,1.5742614129138586,0.26877648344094,0.5160976644437634,0.4051314424838183,34.526645647493076,2.172677637649954,0.7085512150107659,2254441,185495,88160,32655,19192,0.8738336467123499,0.07189887528522917,0.034171297582931096,0.012657256381245633,0.007438924038244255
"2,712",96,2019,26083,5980,64.00234113712375,2578.3312885787677,67250615,16458706,2617899,1838775271.5361295,2049283236.0,0.7645979196808642,0.37698327483715643,1.574261412913811,0.2687764834409361,0.5237489076163141,0.4091713369447804,34.42725146647241,2.176130046390369,0.7046735421538933,2277568,191413,97308,32638,18972,0.8699984223990307,0.07311703010696746,0.03717026516301813,0.012467249500458192,0.007247032830525547
"2,713",96,2020,26753,6527,64.18124712731729,2638.251896983516,70581153,17676847,2703299,1924870966.5916398,2191579382.0,0.8047835289843321,0.37822300459465846,1.6563464593448145,0.2749135133815406,0.5299812326101878,0.4115613339305626,34.91432736515531,2.2610922139573133,0.7002579

In [53]:
df_iptu_subprefeitura["CA médio"].max()

array(8.10014438)

In [54]:
df_iptu_od["CA médio"].max()

array(15.81042331)

In [55]:
df_iptu_sq

#,sq,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos terrenos-lotes,Área Total Ocupada,Área Total Construída,Valor Total dos Terrenos,Valor Total das Construções,CA médio,TO médio,CA médio em lotes condominiais,TO médio em lotes condominiais,CA médio em lotes não condominiais,TO médio em lotes não condominiais,Comprimento Médio da Testada,Número médio de Pavimentos,Fator de obsolecência médio,Residencial,Comercial,Serviços,Industrial,Outros,Percentual de Uso Residencial,Percentual de Uso Comercial,Percentual de Uso Serviços,Percentual de Uso Industrial,Percentual de Uso Outros
0,002083,1995,34,0,nan,167.11764705882354,5682,4577,6684,1287606.0,1113989.0,1.410517477274633,0.8148711515338328,nan,nan,1.410517477274633,0.8148711515338328,8.5,1.911764705882353,0.2647058823529412,370,3222,2612,130,350,0.05535607420706164,0.48204667863554757,0.3907839616995811,0.01944943147815679,0.0523638539796529
1,002083,1996,34,0,nan,167.11764705882354,5682,4794,6684,1873958.0,1484035.0,1.410517477274633,0.8285086578183979,nan,nan,1.410517477274633,0.8285086578183979,7.911764705882353,1.9411764705882353,0.2647058823529412,370,3222,2612,130,350,0.05535607420706164,0.48204667863554757,0.3907839616995811,0.01944943147815679,0.0523638539796529
2,002083,1997,34,0,nan,166.02941176470588,5645,4789,6697,2048070.0,1677578.0,1.4232203574697733,0.8356387825955816,nan,nan,1.4232203574697733,0.8356387825955816,7.882352941176471,1.9411764705882353,0.2647058823529412,270,3335,2612,130,350,0.04031655965357623,0.4979841720173212,0.3900253845005226,0.019411676870240407,0.05226220695833955
3,002083,1998,34,0,nan,166.02941176470588,5645,4789,6697,2160970.0,1770156.0,1.4232203574697733,0.8356387825955816,nan,nan,1.4232203574697733,0.8356387825955816,7.882352941176471,1.9411764705882353,0.20588235294117646,270,3335,2612,130,350,0.04031655965357623,0.4979841720173212,0.3900253845005226,0.019411676870240407,0.05226220695833955
4,002083,1999,34,0,nan,166.02941176470588,5645,4789,6767,2197484.0,1815245.0,1.4381393685524673,0.8356387825955816,nan,nan,1.4381393685524673,0.8356387825955816,7.882352941176471,1.9705882352941178,0.20588235294117646,0,3675,2612,130,350,0.0,0.5430766957292744,0.38599083788975913,0.01921087631151175,0.051721590069454704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"1,233,046",216018,2021,1,0,nan,10631.0,10631,4350,4980,2211248.0,5468040.0,0.4684413507666259,0.40918069795879974,nan,nan,0.4684413507666259,0.40918069795879974,73.5,3.0,0.9100000262260437,0,0,0,4980,0,0.0,0.0,0.0,1.0,0.0
"1,233,047",216018,2022,1,0,nan,10631.0,10631,4350,4980,1913580.0,6538740.0,0.4684413507666259,0.40918069795879974,nan,nan,0.4684413507666259,0.40918069795879974,73.5,3.0,0.8999999761581421,0,0,0,4980,0,0.0,0.0,0.0,1.0,0.0
"1,233,048",252041,2021,22,0,nan,74.4090909090909,1637,1625,2672,514018.0,1812939.0,1.6352701921014825,0.9928844755399779,nan,nan,1.6352701921014825,0.9928844755399779,4.987272761084816,1.9545454545454546,0.7599999904632568,2672,0,0,0,0,1.0,0.0,0.0,0.0,0.0
"1,233,049",252041,2022,22,0,nan,74.4090909090909,1637,1625,2672,844692.0,3324548.0,1.6352701921014825,0.9928844755399779,nan,nan,1.6352701921014825,0.9928844755399779,4.987272761084816,1.9545454545454546,0.7400000095367432,2672,0,0,0,0,1.0,0.0,0.0,0.0,0.0


In [ ]:
gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017', crs='EPSG:31983')


In [ ]:
gdf_od.set_crs(epsg=31983, inplace=True)

In [ ]:
gdf_od = gdf_od[gdf_od.od_municip == '36']
gdf_od['area'] = gdf_od.area
gdf_od.geometry = gdf_od.simplify(tolerance=100)

In [ ]:
gdf_od

In [ ]:
gdf_od.set_crs(epsg=31983).to_crs(epsg=4674, inplace=True)

In [ ]:
gdf_od